In [1]:
from model.data import *
from model.model import *
d = Data()

Using TensorFlow backend.


In [2]:
import os

config = {}
config["weights_file"] = os.getcwd() + '/model/weight'
config["patch_size"] = (64, 64, 64)  # switch to None to train on the whole image
config["patch_gap"] = 16
config["batch_size"] = 10
config["kfold"] = 5

config["input_shape"] = (1, 64, 64, 64)
config["depth"] = 4 # depth of layers for V/Unet
config["n_base_filters"] = 32
config["pool_size"] = (2, 2, 2)  # pool size for the max pooling operations
config["deconvolution"] = True  # if False, will use upsampling instead of deconvolution

config["patience"] = 10  # learning rate will be reduced after this many epochs if the validation loss is not improving
config["early_stop"] = 10  # training will be stopped after this many epochs without the validation loss improving
config["initial_learning_rate"] = 0.00001
config["learning_rate_drop"] = 0.5  # factor by which the learning rate will be reduced
config["n_epochs"] = 1

model = unet_model_3d(input_shape=config["input_shape"],
                              pool_size=config["pool_size"],
                              initial_learning_rate=config["initial_learning_rate"],
                              deconvolution=config["deconvolution"],
                              depth=config["depth"],
                              n_base_filters=config["n_base_filters"])

# model.summary()
counter = 1
for layer in model.layers:
    print(counter, "&", layer.name, "&", layer.output_shape)
    counter += 1


1 & input_1 & (None, 1, 64, 64, 64)
2 & conv3d_1 & (None, 32, 64, 64, 64)
3 & activation_1 & (None, 32, 64, 64, 64)
4 & conv3d_2 & (None, 32, 64, 64, 64)
5 & activation_2 & (None, 32, 64, 64, 64)
6 & max_pooling3d_1 & (None, 32, 32, 32, 32)
7 & conv3d_3 & (None, 64, 32, 32, 32)
8 & activation_3 & (None, 64, 32, 32, 32)
9 & conv3d_4 & (None, 64, 32, 32, 32)
10 & activation_4 & (None, 64, 32, 32, 32)
11 & max_pooling3d_2 & (None, 64, 16, 16, 16)
12 & conv3d_5 & (None, 128, 16, 16, 16)
13 & activation_5 & (None, 128, 16, 16, 16)
14 & conv3d_6 & (None, 128, 16, 16, 16)
15 & activation_6 & (None, 128, 16, 16, 16)
16 & max_pooling3d_3 & (None, 128, 8, 8, 8)
17 & conv3d_7 & (None, 256, 8, 8, 8)
18 & activation_7 & (None, 256, 8, 8, 8)
19 & conv3d_8 & (None, 256, 8, 8, 8)
20 & activation_8 & (None, 256, 8, 8, 8)
21 & max_pooling3d_4 & (None, 256, 4, 4, 4)
22 & conv3d_9 & (None, 512, 4, 4, 4)
23 & activation_9 & (None, 512, 4, 4, 4)
24 & conv3d_10 & (None, 512, 4, 4, 4)
25 & activation_10 & (No

In [4]:
# 2 & conv3d_1 & (None, 32, 64, 64, 64)
# 3 & activation_1 & (None, 32, 64, 64, 64)
# 4 & conv3d_2 & (None, 32, 64, 64, 64)
# 5 & activation_2 & (None, 32, 64, 64, 64)
# 6 & max_pooling3d_1 & (None, 32, 32, 32, 32)
counter = 1
filt = 32
lay = 64
while (filt < 512):
#     print("Contract " + str(counter), "&", "Conv3D + ReLU & (" + str(filt) + ", " + str(lay) + ", " +  str(lay) + ", " + str(lay) + ") \\\\")
    print("Contract " + str(counter), "&", "Conv3D + ReLU & (" + str(filt) + ", " + str(lay) + ", " +  str(lay) + ", " + str(lay) + ") \\\\")
    lay = int(lay / 2)
    print("Contract " + str(counter), "&", "Max pooling & (" + str(filt) + ", " + str(lay) + ", " +  str(lay) + ", " + str(lay) + ") \\\\")
    filt *= 2
    counter += 1    

    
#     2 & Conv3D + ReLU & (64, 64, 64, 64) \\
#     2 & Max pooling & (64, 32, 32, 32) \\

Contract 1 & Conv3D + ReLU & (32, 64, 64, 64) \\
Contract 1 & Max pooling & (32, 32, 32, 32) \\
Contract 2 & Conv3D + ReLU & (64, 32, 32, 32) \\
Contract 2 & Max pooling & (64, 16, 16, 16) \\
Contract 3 & Conv3D + ReLU & (128, 16, 16, 16) \\
Contract 3 & Max pooling & (128, 8, 8, 8) \\
Contract 4 & Conv3D + ReLU & (256, 8, 8, 8) \\
Contract 4 & Max pooling & (256, 4, 4, 4) \\


In [5]:
lay = 8
counter = 4
filt = 256
while (counter >= 1):
    print("Expand " + str(counter), "&", "Deconvolution3D & (" + str(filt) + ", " + str(lay) + ", " +  str(lay) + ", " + str(lay) + ") \\\\")
    print("Expand " + str(counter), "&", "Concatenation & (" + str(filt * 2) + ", " + str(lay) + ", " +  str(lay) + ", " + str(lay) + ") \\\\")
    print("Expand " + str(counter), "&", "Conv3D + ReLU & (" + str(filt) + ", " + str(lay) + ", " +  str(lay) + ", " + str(lay) + ") \\\\")
    filt = int(filt/2)
    lay *= 2
    counter -= 1

Expand 4 & Deconvolution3D & (256, 8, 8, 8) \\
Expand 4 & Concatenation & (512, 8, 8, 8) \\
Expand 4 & Conv3D + ReLU & (256, 8, 8, 8) \\
Expand 3 & Deconvolution3D & (128, 16, 16, 16) \\
Expand 3 & Concatenation & (256, 16, 16, 16) \\
Expand 3 & Conv3D + ReLU & (128, 16, 16, 16) \\
Expand 2 & Deconvolution3D & (64, 32, 32, 32) \\
Expand 2 & Concatenation & (128, 32, 32, 32) \\
Expand 2 & Conv3D + ReLU & (64, 32, 32, 32) \\
Expand 1 & Deconvolution3D & (32, 64, 64, 64) \\
Expand 1 & Concatenation & (64, 64, 64, 64) \\
Expand 1 & Conv3D + ReLU & (32, 64, 64, 64) \\


In [2]:
# # weights-01-0.02-0429-binary-patch.hdf5
# # train_result = h5py.File("./model/h5df_data/train_result-0429-binary-patch.h5", 'r')
# train_result_binary = h5py.File("./model/h5df_data/train_result_binary.h5", 'r')

# d.show_image([train_result_binary["result"][0, :], train_result_binary["result"][1, :], train_result_binary["result"][2, :]])
# print(K.eval(dice_coefficient(train_result_binary["result"][1, :], train_result_binary["result"][2, :])))
# # print(train_result["dice"][()])

interactive(children=(IntSlider(value=64, description='id', max=127), Output()), _dom_classes=('widget-interac…

0.4711363958761286


In [5]:
train_result_dice = h5py.File("./model/h5df_data/train_result_dice.h5", 'r')

d.show_image([train_result_dice["result"][0, :], train_result_dice["result"][1, :], train_result_dice["result"][2, :]])
print(K.eval(dice_coefficient(train_result_dice["result"][1, :], train_result_dice["result"][2, :])))


interactive(children=(IntSlider(value=64, description='id', max=127), Output()), _dom_classes=('widget-interac…

TypeError: Input 'y' of 'Mul' Op has type bool that does not match type float64 of argument 'x'.

In [3]:
train_result_dice_2 = h5py.File("./model/h5df_data/train_result_dice_0.84.h5", 'r')

d.show_image([train_result_dice_2["result"][0, :], train_result_dice_2["result"][1, :], train_result_dice_2["result"][2, :]])
print(K.eval(dice_coefficient(train_result_dice_2["result"][1, :], train_result_dice_2["result"][2, :])))


interactive(children=(IntSlider(value=64, description='id', max=127), Output()), _dom_classes=('widget-interac…

0.8490502189334224


In [17]:
# print((train_result_dice_2["result"][2, :])[0][0].shape)
# img = nib.Nifti1Image(train_result_dice_2["result"][2, :][0][0], np.eye(4))
# nib.save(img, "test.nii")

(128, 256, 256)


In [7]:
def store_nii(arr):
    for i in arr:
        for j in range(3):
            nib.save(nib.Nifti1Image(i["result"][j, :][0][0], np.eye(4)), "dice_" + str(i) + "_" + str(j) + ".nii")
    
store_nii([train_result_dice, train_result_dice_2])

In [11]:
file = [0,0.666024301901755,0.333975698047538,0.6342173402931776,0.3657826595818219,\
1,0.8815179627907102,0.11848203722018623,0.6481558400891895,0.35184415940933184,\
2,0.9235672374277405,0.07643276257755388,0.6888015779389378,0.31119842191141295,\
3,0.9440241258669252,0.055975874132310276,0.6769264744065904,0.3230735257285949,\
4,0.9555893896521793,0.044410610346546496,0.67736474576553,0.3226352540983094,\
0,0.47295483993334,0.03171479501517624,0.6415152951228181,0.01774603456081989,\
1,0.8460512070185201,0.004043289964156918,0.6588427760330703,0.02491079024496241,\
2,0.915436525170012,0.0023498941735349663,0.6816248099683364,0.028382775247403028,\
3,0.945856740332663,0.0015844848736202527,0.6965642363072004,0.0321052242875537,\
4,0.9655379297668429,0.0010783389234943403,0.7055231118234315,0.034446345735856254]

print(len(file))
for i in range(int(len(file)/5)):
    print(str(file[i * 5] + 1) + ' & ', end="")
    for j in range(1, 5):
        if j != 4:
            print("{0:.4f}".format(file[i*5 + j]) + ' & ', end="")
        else:
            print("{0:.4f}".format(file[i*5 + j]) + ' \\\\')
                

50
1 & 0.6660 & 0.3340 & 0.6342 & 0.3658 \\
2 & 0.8815 & 0.1185 & 0.6482 & 0.3518 \\
3 & 0.9236 & 0.0764 & 0.6888 & 0.3112 \\
4 & 0.9440 & 0.0560 & 0.6769 & 0.3231 \\
5 & 0.9556 & 0.0444 & 0.6774 & 0.3226 \\
1 & 0.4730 & 0.0317 & 0.6415 & 0.0177 \\
2 & 0.8461 & 0.0040 & 0.6588 & 0.0249 \\
3 & 0.9154 & 0.0023 & 0.6816 & 0.0284 \\
4 & 0.9459 & 0.0016 & 0.6966 & 0.0321 \\
5 & 0.9655 & 0.0011 & 0.7055 & 0.0344 \\
